In [2]:
import requests
import pandas as pd

In [5]:
df = pd.read_parquet("DatathONS-11/CARGA_ENERGIA_2024.parquet")

In [6]:
df

,id_subsistema,nom_subsistema,din_instante,val_cargaenergiamwmed
0,N,Norte,2024-01-01,6532.986042
1,NE,Nordeste,2024-01-01,11658.740083
2,S,Sul,2024-01-01,10472.403292
3,SE,Sudeste/Centro-Oeste,2024-01-01,35089.386708
4,N,Norte,2024-01-02,7154.334333
...,...,...,...,...
1459,SE,Sudeste/Centro-Oeste,2024-12-30,40639.167375
1460,N,Norte,2024-12-31,7579.586500
1461,NE,Nordeste,2024-12-31,13092.667583
1462,S,Sul,2024-12-31,11997.240667


In [16]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr
from sklearn.feature_selection import mutual_info_regression
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller

def resample_and_merge(df_e, df_c, time_col='timestamp', freq='H'):
    # time_col como datetime
    df_e = df_e.copy()
    df_c = df_c.copy()
    df_e[time_col] = pd.to_datetime(df_e[time_col])
    df_c[time_col] = pd.to_datetime(df_c[time_col])
    df_e = df_e.set_index(time_col).resample(freq).mean()
    df_c = df_c.set_index(time_col).resample(freq).mean()
    df = pd.concat([df_e, df_c], axis=1)
    return df

def rolling_features(df, cols, windows=[1,3,24]):
    for c in cols:
        for w in windows:
            df[f'{c}_rmean_{w}'] = df[c].rolling(w, min_periods=1).mean()
            df[f'{c}_rstd_{w}'] = df[c].rolling(w, min_periods=1).std()
    return df

def test_stationarity(series, alpha=0.05):
    # ADF test: True se estacionária
    series = series.dropna()
    if len(series) < 20:
        return False
    stat, p, *_ = adfuller(series, maxlag=12, autolag='AIC')
    return p < alpha

def compute_pair_metrics(df, x_col, y_col, max_lag=24, corr_method='pearson'):
    results = []
    xs = df[x_col]; ys = df[y_col]
    # raw correlation
    for lag in range(0, max_lag+1):
        if lag == 0:
            xs_l = xs
        else:
            xs_l = xs.shift(lag)
        pair = pd.concat([xs_l, ys], axis=1).dropna()
        if len(pair) < 30:
            continue
        try:
            if corr_method == 'pearson':
                r, p = pearsonr(pair.iloc[:,0], pair.iloc[:,1])
            else:
                r, p = spearmanr(pair.iloc[:,0], pair.iloc[:,1])
        except Exception:
            r, p = np.nan, np.nan
        # mutual info (regression)
        try:
            mi = mutual_info_regression(pair.iloc[:,0].values.reshape(-1,1), pair.iloc[:,1].values, random_state=0)[0]
        except Exception:
            mi = np.nan
        results.append({'x': x_col, 'y': y_col, 'lag': lag, 'corr': r, 'pvalue': p, 'mi': mi, 'n': len(pair)})
    return pd.DataFrame(results)

def granger_pairs(df, x_col, y_col, maxlag=6):
    # Granger: does x -> y ?
    data = df[[y_col, x_col]].dropna()
    if len(data) < maxlag*5:
        return None
    try:
        res = grangercausalitytests(data, maxlag=maxlag, verbose=False)
        # pick best lag by p-value of F-test
        pvals = {lag: res[lag][0]['ssr_ftest'][1] for lag in res}
        best_lag = min(pvals, key=pvals.get)
        return {'x': x_col, 'y': y_col, 'best_lag': best_lag, 'pvalue': pvals[best_lag]}
    except Exception:
        return None

def extract_insights(df, energy_cols, climate_cols, freq='H',
                     max_lag=24, corr_method='pearson', top_k=20):
    # assume df already merged & preprocessed
    records = []
    for xe in energy_cols:
        for yc in climate_cols:
            pair_df = compute_pair_metrics(df, xe, yc, max_lag=max_lag, corr_method=corr_method)
            if pair_df is None or pair_df.empty:
                continue
            # pick top absolute correlation across lags
            pair_df['corr_abs'] = pair_df['corr'].abs()
            best = pair_df.sort_values('corr_abs', ascending=False).iloc[0]
            # granger
            g = granger_pairs(df, xe, yc, maxlag=min(6, max_lag))
            # stability: fraction of windows with same sign corr
            signs = []
            window_size = 24*7  # 1 week
            for start in range(0, len(df)-window_size, window_size):
                seg = df[xe].iloc[start:start+window_size].corr(df[yc].iloc[start:start+window_size])
                signs.append(np.sign(seg) if not np.isnan(seg) else 0)
            stability = np.mean([1 if s == np.sign(best['corr']) else 0 for s in signs]) if signs else 0
            records.append({
                'x': xe, 'y': yc, 'lag': int(best['lag']), 'corr': float(best['corr']),
                'pvalue': float(best['pvalue']) if not np.isnan(best['pvalue']) else np.nan,
                'mi': float(best['mi']) if not np.isnan(best['mi']) else np.nan,
                'n': int(best['n']), 'stability': float(stability),
                'granger_p': g['pvalue'] if g else np.nan, 'granger_lag': g['best_lag'] if g else np.nan
            })
    dfr = pd.DataFrame(records)
    # scoring (normalize components)
    df_nonan = dfr.fillna(0)
    df_nonan['score'] = (df_nonan['corr'].abs() * 0.5 +
                         (1 - df_nonan['pvalue'].clip(0,1)) * 0.2 +
                         (df_nonan['mi'] / (df_nonan['mi'].max() + 1e-9)) * 0.2 +
                         df_nonan['stability'] * 0.1)
    return df_nonan.sort_values('score', ascending=False).head(top_k)

In [20]:
import duckdb
import pandas as pd


con = duckdb.connect(database=':memory:')
print("1")

df_energy = con.execute("""
    SELECT *
    FROM read_parquet('DatathONS-11/CARGA_ENERGIA_2024.parquet')
    WHERE nom_subsistema='Sudeste/Centro-Oeste'
""").df()

df_energy['din_instante'] = pd.to_datetime(df_energy['din_instante'])
df_energy.set_index('din_instante', inplace=True)
df_energy = df_energy[['val_cargaenergiamwmed']] 
print("2")

con.execute("INSTALL sqlite; LOAD sqlite;")  # só na primeira vez
df_clima = con.execute("""
    SELECT *
    FROM sqlite_scan('climate.db', 'climate')
""").df()
print("3")

df_clima['datetime'] = pd.to_datetime(df_clima['data'] + ' ' + df_clima['hora_utc'], errors='coerce')
df_clima = df_clima.set_index('datetime')

cols_to_keep = [c for c in df_clima.columns if df_clima[c].dtype in ['float64', 'int64']]
df_clima = df_clima[cols_to_keep].rename(
    columns=lambda x: x.lower()
                   .replace(' ','_')
                   .replace('(', '')
                   .replace(')','')
                   .replace(',', '_')
                   .replace('.', '')
)
print("Clima pronto:", df_clima.columns.tolist())


df_clima_daily = df_clima.resample('D').mean()
df_energy.index = df_energy.index.tz_localize('UTC')
df_clima_daily.index = df_clima_daily.index.tz_convert('UTC')
df_merged = df_energy.join(df_clima_daily, how='inner')


print("4 - Merge feito")


energy_cols = ['val_cargaenergiamwmed']
climate_cols = list(df_clima_daily.columns)

insights = extract_insights(df_merged, energy_cols, climate_cols, max_lag=24, top_k=10)
print(insights)

1
2


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

3
Clima pronto: ['precipitacao_total__horário_mm', 'pressao_atmosferica_ao_nivel_da_estacao__horaria_mb', 'pressao_atmosferica_maxna_hora_ant_aut_mb', 'pressao_atmosferica_min_na_hora_ant_aut_mb', 'radiacao_global_kj/m²', 'temperatura_do_ar_-_bulbo_seco__horaria_°c', 'temperatura_do_ponto_de_orvalho_°c', 'temperatura_máxima_na_hora_ant_aut_°c', 'temperatura_mínima_na_hora_ant_aut_°c', 'temperatura_orvalho_max_na_hora_ant_aut_°c', 'temperatura_orvalho_min_na_hora_ant_aut_°c', 'umidade_rel_max_na_hora_ant_aut_%', 'umidade_rel_min_na_hora_ant_aut_%', 'umidade_relativa_do_ar__horaria_%', 'vento__direcao_horaria_gr_°_gr', 'vento__rajada_maxima_m/s', 'vento__velocidade_horaria_m/s', 'unnamed:_19']
4 - Merge feito


/opt/miniconda3/envs/ons/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/opt/miniconda3/envs/ons/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/opt/miniconda3/envs/ons/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/opt/miniconda3/envs/ons/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/opt/miniconda3/envs/ons/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/opt/miniconda3/envs/ons/lib/python3.11/site-packages/statsmodels/tsa/stattools.

                        x                                                  y  \
8   val_cargaenergiamwmed              temperatura_mínima_na_hora_ant_aut_°c   
5   val_cargaenergiamwmed         temperatura_do_ar_-_bulbo_seco__horaria_°c   
7   val_cargaenergiamwmed              temperatura_máxima_na_hora_ant_aut_°c   
10  val_cargaenergiamwmed         temperatura_orvalho_min_na_hora_ant_aut_°c   
6   val_cargaenergiamwmed                 temperatura_do_ponto_de_orvalho_°c   
9   val_cargaenergiamwmed         temperatura_orvalho_max_na_hora_ant_aut_°c   
3   val_cargaenergiamwmed         pressao_atmosferica_min_na_hora_ant_aut_mb   
1   val_cargaenergiamwmed  pressao_atmosferica_ao_nivel_da_estacao__horar...   
2   val_cargaenergiamwmed          pressao_atmosferica_maxna_hora_ant_aut_mb   
12  val_cargaenergiamwmed                  umidade_rel_min_na_hora_ant_aut_%   

    lag      corr        pvalue        mi    n  stability     granger_p  \
8     0  0.585113  5.317947e-35  0.390433  3

/opt/miniconda3/envs/ons/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/opt/miniconda3/envs/ons/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/opt/miniconda3/envs/ons/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/opt/miniconda3/envs/ons/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

top_pairs = insights[['x','y','lag']]

heatmap_data = pd.DataFrame(index=range(0,25), columns=top_pairs['y'].values)

for _, row in top_pairs.iterrows():
    x = row['x']
    y = row['y']
    lag = row['lag']
    corrs = []
    for l in range(0,25):
        shifted_x = df_merged[x].shift(l)
        corr = shifted_x.corr(df_merged[y])
        heatmap_data.loc[l, y] = corr

plt.figure(figsize=(12,6))
sns.heatmap(heatmap_data.astype(float), annot=True, cmap='coolwarm', center=0)
plt.title("Heatmap de correlações por lag (0-24 dias)")
plt.xlabel("Variáveis Climáticas")
plt.ylabel("Lag (dias)")
plt.show()

for _, row in top_pairs.iterrows():
    x = row['x']
    y = row['y']
    lag = row['lag']
    plt.figure(figsize=(14,4))
    plt.plot(df_merged.index, df_merged[x], label=x)
    plt.plot(df_merged.index, df_merged[y].shift(lag), label=f"{y} (lag={lag})")
    plt.title(f"Séries Temporais: {x} vs {y} (lag {lag})")
    plt.legend()
    plt.show()

window_size = 7  # em dias
for _, row in top_pairs.iterrows():
    x = row['x']
    y = row['y']
    signs = []
    n_windows = len(df_merged) // window_size
    for start in range(0, len(df_merged)-window_size, window_size):
        seg = df_merged[x].iloc[start:start+window_size].corr(df_merged[y].iloc[start:start+window_size])
        signs.append(np.sign(seg) if not np.isnan(seg) else 0)
    plt.figure(figsize=(10,2))
    plt.bar(range(len(signs)), signs)
    plt.title(f"Estabilidade da correlação: {x} vs {y}")
    plt.ylabel("Sinal da correlação")
    plt.xlabel("Janelas semanais")
    plt.show()

plt.figure(figsize=(8,6))
sns.barplot(data=insights, x='score', y='y', orient='h')
plt.title("Importância das variáveis climáticas (Score agregado)")
plt.xlabel("Score")
plt.ylabel("Variável Climática")
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import pearsonr
import plotly.subplots as sp

def detect_var_type(series):
    n_unique = series.nunique()
    if pd.api.types.is_numeric_dtype(series) and n_unique > 20:
        return 'continuous'
    else:
        return 'categorical'

def normalize_series(df, cols):
    scaler = MinMaxScaler()
    df_norm = df.copy()
    df_norm[cols] = scaler.fit_transform(df[cols])
    return df_norm

def compute_stability(df, x_col, y_col, window_size=24*7):
    signs = []
    for start in range(0, len(df)-window_size, window_size):
        seg = df[x_col].iloc[start:start+window_size].corr(df[y_col].iloc[start:start+window_size])
        signs.append(np.sign(seg) if not np.isnan(seg) else 0)
    return signs


def plot_insight_panel_interactive(df, x_col, y_col, lag=0, metrics=None):
    y_shifted = df[y_col].shift(lag) if lag > 0 else df[y_col]
    df_plot = pd.DataFrame({x_col: df[x_col], y_col: y_shifted}).dropna()

    var_type = detect_var_type(y_shifted)
    if var_type == 'continuous':
        df_plot = normalize_series(df_plot, [x_col, y_col])

    stability_series = compute_stability(df_plot, x_col, y_col)


    fig = sp.make_subplots(
        rows=3, cols=3,
        subplot_titles=[
            'Séries Temporais Normalizadas',
            'Distribuição / Contagem',
            'Scatter + Regressão',
            'Heatmap Correlação por Lag',
            'Boxplot Mensal',
            'Métricas de Insight',
            'Estabilidade por janelas semanais',
            'Scatter múltiplos (Top 3)',
            ''
        ]
    )

    fig.add_trace(go.Scatter(x=df_plot.index, y=df_plot[x_col], name=x_col), row=1, col=1)
    fig.add_trace(go.Scatter(x=df_plot.index, y=df_plot[y_col], name=y_col), row=1, col=1)

    if var_type == 'continuous':
        fig.add_trace(go.Histogram(x=df_plot[y_col], name=y_col), row=1, col=2)
    else:
        vc = df_plot[y_col].value_counts()
        fig.add_trace(go.Bar(x=vc.index, y=vc.values, name=y_col), row=1, col=2)

    fig.add_trace(go.Scatter(x=df_plot[y_col], y=df_plot[x_col], mode='markers', name='scatter'), row=1, col=3)

    lags = min(24, len(df_plot)//2)
    corr_values = [pearsonr(df_plot[x_col], df_plot[y_col].shift(l))[0] for l in range(lags)]
    fig.add_trace(go.Heatmap(z=np.array(corr_values).reshape(-1,1), colorscale='RdBu', showscale=True), row=2, col=1)

    df_box = df_plot.copy()
    df_box['month'] = df_box.index.month
    for m in df_box['month'].unique():
        fig.add_trace(go.Box(y=df_box[df_box['month']==m][y_col], name=str(m)), row=2, col=2)

    if metrics:
        fig.add_trace(go.Bar(x=list(metrics.keys()), y=list(metrics.values())), row=2, col=3)

    fig.add_trace(go.Bar(x=list(range(len(stability_series))), y=stability_series), row=3, col=1)

    top3_cols = [y_col] + df_plot.columns.drop(x_col).tolist()[:2]
    col_idx = 0
    for col in top3_cols:
        fig.add_trace(go.Scatter(x=df_plot[col], y=df_plot[x_col], mode='markers', name=f'{x_col} vs {col}'), row=3, col=2+col_idx%2)
        col_idx += 1

    fig.update_layout(height=1200, width=1600, title_text=f'Painel Interativo: {x_col} vs {y_col} (lag={lag})')
    fig.show()

# -------------------------------
# Aplicar para top 10 insights
# -------------------------------
top10_insights = insights.head(10)

for idx, row in top10_insights.iterrows():
    metrics = {
        'corr': row['corr'],
        'mi': row['mi'],
        'granger_p': row['granger_p'],
        'stability': row['stability']
    }
    plot_insight_panel_interactive(df_merged, row['x'], row['y'], lag=int(row['lag']), metrics=metrics)